In [1]:
%matplotlib widget
from smpr3d.core import Sparse4DData, Metadata4D
from numpy.fft import fftshift, fft2
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
from matplotlib_scalebar.scalebar import ScaleBar
from pathlib import Path
from smpr3d.util import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tifffile import imsave, imwrite

scan_number = 358

# base_path = Path('/home/philipp/nvme/insync_berkeley/2020-09-22/')
base_path = Path('/home/philipp/nvme/2021-04-30_tomo/')

adfpath = base_path / 'adf'
adfpath = Path('/home/philipp/team05/PhilippPelz/2021-04-28/')

sparse_path = base_path / '4d/'

results_path = base_path / 'results/'
results_path_scan = results_path / f'scan{scan_number}'
results_path_vadf = results_path / f'vadf'
results_path_adf = results_path / f'adf'
results_path_abf = results_path / f'abf'
results_path_bf = results_path / f'bf'
results_path_dpc = results_path / f'dpc'
results_path_ptychography = results_path / f'dpc'

if not results_path.exists(): results_path.mkdir()
if not results_path_scan.exists(): results_path_scan.mkdir()
if not results_path_vadf.exists(): results_path_vadf.mkdir()
if not results_path_adf.exists(): results_path_adf.mkdir()
if not results_path_abf.exists(): results_path_abf.mkdir()
if not results_path_bf.exists(): results_path_bf.mkdir()
if not results_path_dpc.exists(): results_path_dpc.mkdir()
if not results_path_ptychography.exists(): results_path_ptychography.mkdir()
    
filename4d = sparse_path / f'data_scan{scan_number}_th4.0_electrons.h5'
filenameadf = adfpath / f'scan{scan_number}.dm4'

In [2]:
# from numba import jit
# import h5py

# @jit
# def unragged_frames_size(frames):
#         mm = 0
#         for ev in frames:
#             if ev.shape[0] > mm:
#                 mm = ev.shape[0]
#         return mm

# @jit(nopython=True)
# def make_unragged_frames(frames, scan_dimensions):
#     unragged_frame_size = unragged_frames_size(frames.ravel())
#     fr_full = np.zeros((frames.ravel().shape[0], unragged_frame_size), dtype=np.int32)
#     fr_full[:] = np.iinfo(fr_full.dtype).max
#     for ii, ev in enumerate(frames.ravel()):
#         fr_full[ii, :ev.shape[0]] = ev
#     fr_full_4d = fr_full.reshape((*scan_dimensions, fr_full.shape[1]))
#     fr_full_4d = fr_full_4d[:, :-1, :] 
#     return fr_full_4d
    
# def from_4Dcamera_file(filename):
#     with h5py.File(filename, 'r') as f0:
#         frames = f0['electron_events/frames'][:]
#         scan_dimensions = (f0['electron_events/scan_positions'].attrs['Ny'],
#                            f0['electron_events/scan_positions'].attrs['Nx'])
#         frame_dimensions = np.array((576, 576))

#     d = Sparse4DData()
#     d.indices = np.ascontiguousarray(make_unragged_frames(frames.ravel(), scan_dimensions))
#     d.scan_dimensions = np.array(d.indices.shape[:2])
#     d.frame_dimensions = frame_dimensions
#     d.counts = np.zeros(d.indices.shape, dtype=np.bool)
#     d.counts[d.indices != np.iinfo(d.indices.dtype).max] = 1

#     return d 

In [3]:
da = []
mda = []

results_path_scan = results_path / f'scan{scan_number}'
if not results_path_scan.exists():
    results_path_scan.mkdir()
filename4d = sparse_path / f'data_scan{scan_number}_th4.0_electrons.h5'
# filename4d = sparse_path / f'data_scan{scan_number}_th4.5_electrons.h5'
# filename4d = sparse_path / f'data_scan{scan_number}_th4_electrons.h5'
filenameadf = adfpath / f'scan{scan_number}.dm4'
# filenameadf = adfpath / f'scan{scan_number}_ADF.dm4'
d = Sparse4DData.from_4Dcamera_file(filename4d)
print('1')
metadata = Metadata4D.from_dm4_file(filenameadf)
metadata.alpha_rad = 25e-3
metadata.alpha_rad = 20e-3
metadata.rotation_deg = 0
metadata.wavelength =  wavelength(metadata.E_ev)  
mda.append(metadata)

center, radius = d.determine_center_and_radius(manual=False, size=200) 
# center = [292.0,169.0]
radius = 120
print(center,radius)
# center[0] -= 2
# center[1] -= 3
print('2')
d.crop_symmetric_center_(center)
print('3')
s = d.sum_diffraction()
print('4')

da.append(d)

f,ax = plt.subplots(figsize=(4,4))
imax = ax.imshow(s)
ax.set_title(f'Scan {scan_number} sum after cropping')
plt.colorbar(imax)
plt.tight_layout()
f.savefig(results_path_scan /f'sum_crop.png')

1
[290.03576852 167.0052303 ] 120
2
old frames shape: (512, 512, 1146)
new frames shape: (512, 512, 1146)
old frames frame_dimensions: [576 576]
new frames frame_dimensions: [334 334]
3
4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
abf = d.virtual_annular_image(radius/2, radius, d.frame_dimensions/2)
bf = d.virtual_annular_image(0, radius/2, d.frame_dimensions/2)
eabf = abf - bf
adf = d.virtual_annular_image(radius, d.frame_dimensions[0]/2, d.frame_dimensions/2)

bf[bf==0] = bf.mean()
abf[abf==0] = abf.mean()

In [5]:

w,h = matplotlib.figure.figaspect(bf)
fig, ax = plt.subplots(dpi=150,figsize=(w, h))
im = ax.imshow(abf, cmap= plt.cm.get_cmap('bone'))
ax.set_title(f'Scan {scan_number} ABF')
ax.set_xticks([])
ax.set_yticks([])
ax.add_artist(ScaleBar(metadata.scan_step[0]/10,'nm'))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.tight_layout()
fig.savefig(results_path_abf /f'scan{scan_number}_abf.png')
fig.savefig(results_path_scan /f'scan{scan_number}_abf.png')
fig.savefig(results_path_adf /f'scan{scan_number}_abf.pdf')
fig, ax = plt.subplots(dpi=150,figsize=(w, h))
im = ax.imshow(bf, cmap= plt.cm.get_cmap('bone'))
ax.set_title(f'Scan {scan_number} BF')
ax.set_xticks([])
ax.set_yticks([])
ax.add_artist(ScaleBar(metadata.scan_step[0]/10,'nm'))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.tight_layout()
fig.savefig(results_path_bf /f'scan{scan_number}_bf.png')
fig.savefig(results_path_scan /f'scan{scan_number}_bf.png')
fig.savefig(results_path_adf /f'scan{scan_number}_bf.pdf')
fig, ax = plt.subplots(dpi=150,figsize=(w, h))
im = ax.imshow(adf, cmap= plt.cm.get_cmap('bone'))
ax.set_title(f'Scan {scan_number} ADF')
ax.set_xticks([])
ax.set_yticks([])
ax.add_artist(ScaleBar(metadata.scan_step[0]/10,'nm'))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.tight_layout()

# imwrite(results_path_scan /f'scan{scan_number}_abf.tif', abf.astype('float32'), imagej=True, resolution=(1./(metadata.scan_step[0]/10), 1./(metadata.scan_step[1]/10)),
#             metadata={'spacing': 1 / 10, 'unit': 'nm', 'axes': 'ZYX'})
# imwrite(results_path_scan /f'scan{scan_number}_bf.tif', bf.astype('float32'), imagej=True, resolution=(1./(metadata.scan_step[0]/10), 1./(metadata.scan_step[1]/10)),
#             metadata={'spacing': 1 / 10, 'unit': 'nm', 'axes': 'ZYX'})
# imwrite(results_path_scan /f'scan{scan_number}_eabf.tif', eabf.astype('float32'), imagej=True, resolution=(1./(metadata.scan_step[0]/10), 1./(metadata.scan_step[1]/10)),
#             metadata={'spacing': 1 / 10, 'unit': 'nm', 'axes': 'ZYX'})
# imwrite(results_path_scan /f'scan{scan_number}_adf', adf.astype('float32'), imagej=True, resolution=(1./(metadata.scan_step[0]/10), 1./(metadata.scan_step[1]/10)),
#             metadata={'spacing': 1 / 10, 'unit': 'nm', 'axes': 'ZYX'})

fig.savefig(results_path_adf /f'scan{scan_number}_adf.png')
fig.savefig(results_path_scan /f'scan{scan_number}_adf.png')
fig.savefig(results_path_adf /f'scan{scan_number}_adf.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
dwell_time = 1/87e3
detector_to_real_fluence_80kv = 1#1/0.56

fluence = d.fluence(metadata.scan_step[0]) * detector_to_real_fluence_80kv
flux = d.flux(metadata.scan_step[0], dwell_time) * detector_to_real_fluence_80kv


area = np.pi * radius **2

# print(f"E               = {metadata.E_ev/1e3}             keV")
# print(f"λ               = {metadata.wavelength * 1e2:2.2}   pm")
print(f'radius: {radius}')
print(f'[min,max] electrons to avoid coincidence losses: [{area//100},{area//40}]')
print(f"dR              = {metadata.scan_step} Å")
print(f"scan       size = {d.scan_dimensions}")
print(f"detector   size = {d.frame_dimensions}")
print(f"scan       FOV  = {d.scan_dimensions*metadata.scan_step/10} nm")
print(f"fluence         ~ {fluence} e/Å^2")
print(f"flux            ~ {flux} e/Å^2/s")

radius: 120
[min,max] electrons to avoid coincidence losses: [452.0,1130.0]
dR              = [0.22213781 0.22213781] Å
scan       size = [512 512]
detector   size = [334 334]
scan       FOV  = [11.37345581 11.37345581] nm
fluence         ~ 19049.463345044624 e/Å^2
flux            ~ 6322.110408854989 e/Å^2/s


In [7]:
alpha_max_factor = 1.2
alpha_max_factor = 1.05
dssb = d.crop_symmetric_center(d.frame_dimensions/2, radius*alpha_max_factor)
metadata.k_max = metadata.alpha_rad * alpha_max_factor / metadata.wavelength

s = dssb.sum_diffraction()

f,ax = plt.subplots(figsize=(4,4))
imax = ax.imshow(s)
plt.colorbar(imax)

old frames shape: (512, 512, 1146)
new frames shape: (512, 512, 1146)
old frames frame_dimensions: [334 334]
new frames frame_dimensions: [252 252]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
metadata

Metadata4D(scan_step=array([0.22213781, 0.22213781], dtype=float32), pixel_step=array([0, 0]), k_max=0.5029076208645908, alpha_rad=0.02, rotation_deg=0, E_ev=80000.0, wavelength=0.041757171951176904, aberrations=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [9]:
import time
from smpr3d.util import get_qx_qy_1D, ZernikeProbeSingle, fourier_coordinates_2D, imsave
from ipywidgets import AppLayout, FloatSlider, GridspecLayout, VBox, Tab, Box, HBox, IntSlider
import ipywidgets as widgets
from matplotlib_scalebar.scalebar import ScaleBar
from numpy.fft import fftshift
from math import sin, cos
import time
import cupy as cp
import matplotlib.pyplot as plt
from cupy.cuda import Device
import cupyx.scipy.fft as fft
from IPython.display import display
from skimage.filters import gaussian
from matplotlib.patches import Rectangle
plt.ioff()

out0 = widgets.Output(layout={'border': '1px solid black'})

class InteractiveSSB:
    def __init__(self, sparse_data, slice_image, radius, meta, window_size=128):
        self.out = widgets.Output(layout={'border': '1px solid black'})
        self.meta = meta
        self.data = sparse_data
        self.ssb_size = 32
        self.slic = np.s_[0:sparse_data.scan_dimensions[0],0:sparse_data.scan_dimensions[1]]
        self.rmax = sparse_data.frame_dimensions[-1] // 2
        self.alpha_max = self.rmax / radius * meta.alpha_rad
        self.slice_image = slice_image
        self.window_center = sparse_data.scan_dimensions//2
        self.window_size = sparse_data.scan_dimensions[0] 

        r_min = meta.wavelength / (2 * self.alpha_max)
        r_min = np.array([r_min, r_min])
        self.k_max = [self.alpha_max / meta.wavelength, self.alpha_max / meta.wavelength]
        self.r_min = np.array(r_min)
        self.dxy = np.array(meta.scan_step)
        
        self.margin = sparse_data.frame_dimensions // 2
        self.rotation_deg = 0.0
        
        self.out.append_stdout(f"alpha_max       = {self.alpha_max * 1e3:2.2f} mrad\n")
        self.out.append_stdout(f"E               = {meta.E_ev/1e3} keV\n")
        self.out.append_stdout(f"λ               = {meta.wavelength * 1e2:2.2}   pm\n")
        self.out.append_stdout(f"dR              = {self.dxy[0]:2.2f}             Å\n")
        self.out.append_stdout(f"dK              = {self.k_max[0]:2.2f}          Å^-1\n")
        self.out.append_stdout(f"scan       size = {sparse_data.scan_dimensions}\n")
        self.out.append_stdout(f"detector   size = {sparse_data.frame_dimensions}\n")
        
        self.rotation_slider = widgets.FloatSlider(
            value=0,
            min=-180,
            max=180,
            step=0.1,
            description='STEM rotation angle:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )
        self.rotation_slider.observe(self.rotation_changed, 'value')
        
        self.window_size_slider = widgets.IntSlider(
            value=sparse_data.scan_dimensions[0],
            min=2,
            max=np.max(sparse_data.scan_dimensions),
            step=2,
            description='Window size',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        self.window_size_slider.observe(self.window_size_slider_changes, 'value')
        
        self.aberration_text = widgets.HTML(
                value="1",
                placeholder='',
                description='',
            )
        
        
        self.gs = GridspecLayout(4,9)
        self.Cslider_box = VBox(width=50)
        self.scale_slider_box = VBox()
        children= []
        sliders =  []

        self.probe_output = widgets.Output()
        self.overlaps_output = widgets.Output()
        
        qn = fourier_coordinates_2D([128,128],r_min)
        q = th.as_tensor(qn).cuda()
        alpha = q * meta.wavelength
        self.probe_gen = ZernikeProbeSingle(q, meta.wavelength, fft_shifted=False)
        self.A = np.linalg.norm(qn,axis=0) * meta.wavelength < meta.alpha_rad
        self.A = gaussian(self.A,1)
                
        self.C = cp.zeros((12,))
        self.C_names = ['C1','C12a' ,'C12b','C21a','C21b','C23a','C23b','C3','C32a','C32b','C34a','C34b']
        self.C_min = [-100,-20,-20,-50,-50,-50,-50,-20,-20,-20,-20,-20]
        self.C_max = [100,20,20,50,50,50,50,20,20,20,20,20]
        self.C_multiplier = [1e1,1e1,1e1,1e1,1e1,1e1,1e4,1e4,1e4,1e4,1e4]
        
        Psi = self.probe_gen(th.tensor(self.C.get()).cuda(),th.tensor(self.A).cuda())
        self.phases = th.angle(th.fft.fftshift(self.probe_gen(th.tensor(self.C.get()).cuda(),th.tensor(np.ones_like(self.A)).cuda())))
        self.Psi_shifted = th.fft.fftshift(Psi)
        self.psi = th.fft.fftshift(th.fft.ifft2(Psi))
        
        with self.probe_output:
            self.probe_figure = plt.figure(constrained_layout=True,figsize=(9,3))
            gs1 = self.probe_figure.add_gridspec(1, 3, wspace=0.05,hspace=0.05)
            self.f1ax0 = self.probe_figure.add_subplot(gs1[0])
            self.f1ax1 = self.probe_figure.add_subplot(gs1[1])
            self.f1ax2 = self.probe_figure.add_subplot(gs1[2])
            
            self.f1ax0.set_title('Probe  (real space)')
            self.f1ax1.set_title('Probe  (Fourier space)')
            self.f1ax2.set_title('Phase profile (Fourier space)')
            
            self.probe_realspace_imax = self.f1ax0.imshow(imsave(self.psi.cpu().numpy()))
            self.probe_fourier_imax = self.f1ax1.imshow(imsave(self.Psi_shifted.cpu().numpy()))
            self.probe_phases_imax = self.f1ax2.imshow(self.phases.cpu().numpy())
            #,cmap=plt.cm.get_cmap('Greys')

            self.f1ax0.set_xticks([])
            self.f1ax0.set_yticks([])
            self.f1ax1.set_xticks([])
            self.f1ax1.set_yticks([])
            self.f1ax2.set_xticks([])
            self.f1ax2.set_yticks([])
            
        self.overlap_figure_axes = []
        with self.overlaps_output:
            self.overlap_figure = plt.figure(constrained_layout=True,figsize=(9,9))
            gs1 = self.overlap_figure.add_gridspec(3, 3, wspace=0.05,hspace=0.05)
            for ggs in gs1:
                f3_ax1 = self.overlap_figure.add_subplot(ggs)
                imax2 = f3_ax1.imshow(np.zeros((40,40)))
                f3_ax1.set_xticks([])
                f3_ax1.set_yticks([])
                self.overlap_figure_axes.append(imax2)
                
        self.plot_box = VBox(children =[self.probe_figure.canvas,self.overlap_figure.canvas])    
        self.recon_fig, self.recon_axes = plt.subplots(figsize=(9,9))
        self.recon_img = self.recon_axes.imshow(np.zeros(sparse_data.scan_dimensions), cmap=plt.get_cmap('bone'))
        self.recon_axes.set_xticks([])
        self.recon_axes.set_yticks([])
        scalebar = ScaleBar(meta.scan_step[0]/10,'nm') # 1 pixel = 0.2 meter
        self.recon_axes.add_artist(scalebar)

        for i, (name, mins, maxs, multiplier) in enumerate(zip(self.C_names, self.C_min, self.C_max, self.C_multiplier)):
            s = FloatSlider(description=name,
                           min=mins, max = maxs)
            s.observe(self.create_function(f'slider_changed_{i}', i, multiplier), names='value')
            sliders.append(s)
            children.append(s)

        self.Cslider_box.children = children + [self.aberration_text]

        self.gs[:2,0] = self.Cslider_box
        self.gs[2:,0] = self.scale_slider_box
        self.gs[:,1:5] = self.plot_box
        self.gs[:,5:9] = self.recon_fig.canvas
        
        self.first_time_calc = True
    def __del__(self):
        del self.recon_fig
        del self.overlap_figure
        del self.probe_figure
        
    def rotation_changed(self, change):
        self.rotation_deg = change['new']
        
    @out0.capture()
    def update_slice(self):
        y0 = self.window_center[0] - self.window_size[0]//2
        y1 = self.window_center[0] + self.window_size[0]//2
        x0 = self.window_center[1] - self.window_size[1]//2
        x1 = self.window_center[1] + self.window_size[1]//2
        
        y0 = int(y0 if y0 > 0 else 0)
        y1 = int(y1 if y1 <= self.data.scan_dimensions[0] else self.data.scan_dimensions[0])
        x0 = int(x0 if x0 > 0 else 0)
        x1 = int(x1 if x1 <= self.data.scan_dimensions[1] else self.data.scan_dimensions[1])
        
        if y0 == 0:
            y1 = (y1//2)*2
            self.window_size[0] = y1-y0
            
        if x0 == 0:
            x1 = (x1//2)*2
            self.window_size[1] = x1-x0
            
        if y1 == self.data.scan_dimensions[0] and y0 % 2 > 0:
            y0 -= 1
            self.window_size[0] = self.window_size[0] + 1
            
        if x1 == self.data.scan_dimensions[0] and x0 % 2 > 0:
            x0 -= 1
            self.window_size[1] = self.window_size[1] + 1
        
#         y1 = y1 if y0 > 0 else (y1 // 2) * 2
#         x1 = x1 if x0 > 0 else (x1 // 2) * 2
        
        self.slic = np.s_[y0:y1,x0:x1]
        self.out.append_stdout(f"slice {y0},{y1},{x0},{x1}\n")
        
        self.slice_rect.set_xy((x0,y0))
        self.slice_rect.set_height(y1-y0)
        self.slice_rect.set_width(x1-x0)
        
        self.G = self._get_G(self.ssb_size)
        self.Gabs = cp.sum(cp.abs(self.G), (2, 3))
        
        sh = np.array(self.Gabs.shape)
        mask = ~np.array(fftshift(sector_mask(sh, sh // 2, sh[0]/30, (0, 360))))
        mask[:,-1] = 0
        mask[:,0] = 0
        mask[:,1] = 0
        
        gg = np.log10(self.Gabs.get()+1)
        gg[~mask] = gg.mean()
        gg = fftshift(gg)
        gg1 = gg#[self.margin[0]:-self.margin[0],self.margin[1]:-self.margin[1]]
        self.imax_power_spectrum.set_data(gg1)
        self.imax_power_spectrum.set_clim(gg1.min(),gg1.max())
        
        self.fig_select_slice.canvas.draw()
        self.fig_select_slice.canvas.flush_events()
        self.fig_power_spectrum.canvas.draw()
        self.fig_power_spectrum.canvas.flush_events()
    
    @out0.capture()
    def window_size_slider_changes(self, change):
        self.window_size = [change['new'],change['new']]
        self.update_slice()
            
    @out0.capture()
    def fig_select_slice_onclick(self, event):
        ix, iy = event.xdata, event.ydata
#         self.out.append_stdout(f"clicked ix:{ix},iy:{iy}\n")
        self.window_center = [iy, ix]
        self.update_slice()
        
    @out0.capture()
    def _get_G(self, size):
        bin_factor = int(np.min(np.floor(self.data.frame_dimensions/size)))
        start = time.perf_counter()
        dc = self.dc[self.slic]
        
        M = cp.array(dc, dtype=cp.complex64)
        ny, nx, nky, nkx = M.shape
        start = time.perf_counter()
        G = fft.fft2(M, axes=(0, 1), overwrite_x=True)
        G /= cp.sqrt(np.prod(G.shape[:2]))
        
        return G
    
    @out0.capture()
    def _get_G_full(self, size):
        bin_factor = int(np.min(np.floor(self.data.frame_dimensions/size)))
        start = time.perf_counter()
        data = self.data.slice(self.slic)
        self.dc = data.to_dense(bin_factor)
        self.out.append_stdout(f"Bin by {bin_factor} for ssb took {time.perf_counter() - start:2.2g}s\n")
        self.out.append_stdout(f"Data shape is {self.dc.shape}\n")
        
        self.Qy1d, self.Qx1d = get_qx_qy_1D(data.scan_dimensions, self.dxy, cp.float32, fft_shifted=False)
        self.Ky, self.Kx = get_qx_qy_1D(self.dc.shape[-2:], self.r_min, cp.float32, fft_shifted=True)
        
        self.Psi_Qp = cp.zeros(data.scan_dimensions, dtype=np.complex64)
        self.Psi_Qp_left_sb = cp.zeros(data.scan_dimensions, dtype=np.complex64)
        self.Psi_Qp_right_sb = cp.zeros(data.scan_dimensions, dtype=np.complex64)
        self.Psi_Rp = cp.zeros(data.scan_dimensions, dtype=np.complex64)
        self.Psi_Rp_left_sb = cp.zeros(data.scan_dimensions, dtype=np.complex64)
        self.Psi_Rp_right_sb = cp.zeros(data.scan_dimensions, dtype=np.complex64)
        
        M = cp.array(self.dc, dtype=cp.complex64)
        ny, nx, nky, nkx = M.shape
        start = time.perf_counter()
        G = fft.fft2(M, axes=(0, 1), overwrite_x=True)
        G /= cp.sqrt(np.prod(G.shape[:2]))
        
        self.out.append_stdout(f"FFT along scan coordinate took {time.perf_counter() - start:2.2g}s\n")
        return G
    
    @out0.capture()
    def update_variables(self):
        self.Psi_Qp[:] = 0
        self.Psi_Qp_left_sb[:] = 0
        self.Psi_Qp_right_sb[:] = 0
            
        eps=1e-3
        start = time.perf_counter()
        single_sideband_reconstruction(
                self.G,
                self.Qx1d,
                self.Qy1d,
                self.Kx,
                self.Ky,
                self.C,
                np.deg2rad(self.rotation_deg),
                self.meta.alpha_rad,
                self.Psi_Qp,
                self.Psi_Qp_left_sb,
                self.Psi_Qp_right_sb,
                eps,
                self.meta.wavelength,
            )
        m = 2

        self.Psi_Rp[:] = fft.ifft2(self.Psi_Qp, norm="ortho")
        self.Psi_Rp_left_sb[:] = fft.ifft2(self.Psi_Qp_left_sb, norm="ortho")
        self.Psi_Rp_right_sb[:] = fft.ifft2(self.Psi_Qp_right_sb, norm="ortho")
        
        self.Gamma = disk_overlap_function(self.Qx_max1d, self.Qy_max1d, self.Kx, self.Ky, self.C, np.deg2rad(self.rotation_deg), self.meta.alpha_rad, self.meta.wavelength)
        
        Psi = self.probe_gen(th.tensor(self.C.get()).cuda(),th.tensor(self.A).cuda())
        self.phases = th.angle(th.fft.fftshift(self.probe_gen(th.tensor(self.C.get()).cuda(),th.tensor(self.A).cuda())))
        self.Psi_shifted = th.fft.fftshift(Psi)
        self.psi = th.fft.fftshift(th.fft.ifft2(Psi))
            
    @out0.capture()
    def update_gui(self):
        gg = self.Gamma * self.G_max

        m=10
        img = np.angle(self.Psi_Rp_left_sb.get()[m:-m,m:-m])
        self.recon_img.set_data(img)
        self.recon_img.set_clim(img.min(),img.max())

        for ax, g in zip(self.overlap_figure_axes,gg):
            ax.set_data(imsave(g.get()))
        
        self.probe_realspace_imax.set_data(imsave(self.psi.cpu().numpy()))
        self.probe_fourier_imax.set_data(imsave(self.Psi_shifted.cpu().numpy()))
        self.probe_phases_imax.set_data(self.phases.cpu().numpy())
        self.probe_phases_imax.set_clim(self.phases.min(),self.phases.max())
        
        self.recon_fig.canvas.draw()
        self.overlap_figure.canvas.draw()
        self.probe_figure.canvas.draw()
        
        self.recon_fig.canvas.flush_events()
        self.overlap_figure.canvas.flush_events()
        self.probe_figure.canvas.flush_events()
    
    @out0.capture()
    def create_function(self, name, i, multiplier):
        def func1(change):
            self.C[i] = change['new']*multiplier
            w = change['new']*multiplier
#             self.out.append_stdout(f"C[{i}] = {self.C[i]}\n")
            
            self.update_variables()
            self.update_gui()
            
        func1.__name__ = name
        return func1
    
    
    @out0.capture()
    def selected_index_changed(self, change):
        w = change['new']
        if w == 1:
#             if self.first_time_calc:
            n_fit = 9
            self.G = self._get_G_full(self.ssb_size)
            self.Gabs = cp.sum(cp.abs(self.G), (2, 3))
            self.out.append_stdout(f"self.G.shape {self.G.shape}\n")

            sh = np.array(self.Gabs.shape)
            mask = ~np.array(fftshift(sector_mask(sh, sh // 2, 15, (0, 360))))
            mask[:,-1] = 0
            mask[:,0] = 0
            mask[:,1] = 0

            gg = self.Gabs.get()
            gg[~mask] = gg.mean()

            inds = np.argsort((gg).ravel())
            strongest_object_frequencies = np.unravel_index(inds[-1 - n_fit:-1], self.G.shape[:2])

            self.out.append_stdout(f"strongest_object_frequencies: {strongest_object_frequencies}\n")

            self.G_max = self.G[strongest_object_frequencies]
            self.Qy_max1d = self.Qy1d[strongest_object_frequencies[0]]
            self.Qx_max1d = self.Qx1d[strongest_object_frequencies[1]]
                
            self.update_variables()
            self.update_gui()
            
        elif w==2:
            data = dssb 
            bright_field_radius = radius

            meta = metadata

            defocus_list_nm = np.linspace(-300,300,60,endpoint=True)  + 0
            print(f'defocus_list_nm: {defocus_list_nm}')
            defocal = []
            defocal_right = []
            defocal_left = []

            for df in defocus_list_nm:
                print(df)
                C = xp.zeros((12,))
                C[0] = df
                Psi_Qp = cp.zeros((ny, nx), dtype=G.dtype)
                Psi_Qp_left_sb = cp.zeros((ny, nx), dtype=np.complex64)
                Psi_Qp_right_sb = cp.zeros((ny, nx), dtype=np.complex64)
                start = time.perf_counter()
                eps = 1e-3
                single_sideband_reconstruction(
                        self.G,
                        self.Qx1d,
                        self.Qy1d,
                        self.Kx,
                        self.Ky,
                        self.C,
                        np.deg2rad(self.rotation_deg),
                        self.meta.alpha_rad,
                        self.Psi_Qp,
                        self.Psi_Qp_left_sb,
                        self.Psi_Qp_right_sb,
                        eps,
                        self.meta.wavelength,
                    )
                self.Psi_Rp[:] = fft.ifft2(self.Psi_Qp, norm="ortho")
                self.Psi_Rp_left_sb[:] = fft.ifft2(self.Psi_Qp_left_sb, norm="ortho")
                self.Psi_Rp_right_sb[:] = fft.ifft2(self.Psi_Qp_right_sb, norm="ortho")

                defocal.append(Psi_Rp.get())
                defocal_right.append(Psi_Rp_right_sb.get())
                defocal_left.append(Psi_Rp_left_sb.get())

            ssb_defocal = np.array(defocal)
            ssb_defocal_right = np.array(defocal_right)
            ssb_defocal_left = np.array(defocal_left)
            
#             fig, ax = plt.subplots(dpi=150)

#             var1 = np.var(ssb_defocal[...,m:-m,m:-m],(1,2))
#             var2 = np.var(ssb_defocal_right[...,m:-m,m:-m],(1,2))

#             for i, v in enumerate([var1,var2,var3]):
#                 ax.scatter(np.arange(len(v)), v, label=f'{i}', s=3)
#                 ax.plot(v, label=f'{i}')
#             plt.legend()
            
#             mxs = np.argmax(var1)
#             best_df = defocus_list_nm[mxs]
#             mxs2 = np.argmax(var2)
#             best_df2 = defocus_list_nm[mxs2]
            
            self.out.append_stdout(f'best_df ssb_defocal: {mxs}:{best_df}\n')
            self.out.append_stdout(f'best_df ssb_defocal_right: {mxs2}:{best_df2}\n')
    
    def show(self):
        n_fit = 49
        ranges=[360, 30]
        partitions = [144, 120]
        ranges = [360]
        partitions=[360]
        manual_frequencies=None
        manual_frequencies = None
        
        bin_factor = int(np.min(np.floor(self.data.frame_dimensions/12)))
        data = self.data.slice(self.slic)
        self.dc = data.to_dense(bin_factor)
        
        M = cp.array(self.dc, dtype=cp.complex64)
        ny, nx, nky, nkx = M.shape
        self.G = fft.fft2(M, axes=(0, 1), overwrite_x=True)
        self.G /= cp.sqrt(np.prod(self.G.shape[:2]))
        self.Gabs = cp.sum(cp.abs(self.G), (2, 3))
        
        sh = np.array(self.Gabs.shape)
        mask = ~np.array(fftshift(sector_mask(sh, sh // 2, 15, (0, 360))))
        mask[:,-1] = 0
        mask[:,0] = 0
        mask[:,1] = 0
        
        gg = np.log10(self.Gabs.get()+1)
        gg[~mask] = gg.mean()
        gg = fftshift(gg)
        gg1 = gg[self.margin[0]:-self.margin[0],self.margin[1]:-self.margin[1]]
        
        self.fig_select_slice, self.ax_select_slice = plt.subplots(1,1,figsize=(8,8))
        self.fig_select_slice.canvas.mpl_connect('button_press_event', self.fig_select_slice_onclick)
        self.imax_select_slice = self.ax_select_slice.imshow(self.slice_image, cmap=plt.cm.get_cmap('magma'))
        self.slice_rect = Rectangle((self.slic[0].start,self.slic[1].start), self.window_size, self.window_size, fill=False, lw=3,ls='--')
        self.ax_select_slice.add_patch(self.slice_rect)        
        
        self.fig_power_spectrum, self.ax_power_spectrum = plt.subplots(1,1,figsize=(8,8))
        self.imax_power_spectrum = self.ax_power_spectrum.imshow(gg1, cmap=plt.cm.get_cmap('magma'))
        

        best, thetas, intensities = find_rotation_angle_with_double_disk_overlap(self.G, self.meta.wavelength, self.r_min, self.dxy, self.meta.alpha_rad, mask=cp.array(mask), ranges=ranges,
                                                                  partitions=partitions, n_fit=n_fit, verbose=False,
                                                                  manual_frequencies=manual_frequencies, aberrations=self.C)

        self.out.append_stdout(f"Best rotation angle: {np.rad2deg(thetas[best])}\n")
        
        fig, ax = plt.subplots()
        ax.scatter(np.rad2deg(thetas),intensities)
        ax.set_xlabel('STEM rotation [degrees]')
        ax.set_ylabel('Integrated G amplitude over double overlap')
                      
        rot_box = VBox([fig.canvas,self.rotation_slider])
        
        window_size_box = VBox([self.window_size_slider, self.fig_select_slice.canvas])
        
        canvas_box = HBox([window_size_box, self.fig_power_spectrum.canvas,  rot_box])
        gsl1 = VBox([canvas_box])
        gsl2 = GridspecLayout(1,1)
        
        tab_contents = ['P0', 'P1','P2']
        children = [gsl1,self.gs]
        tab = widgets.Tab()
        tab.children = children
        tab.titles = ['fft','00','00']
        
        tab.observe(self.selected_index_changed, 'selected_index')
        
        gsl00 = GridspecLayout(20,20)
        gsl00[3:,:3] = self.out
        gsl00[3:,3:] = tab
        gsl00[:3,:] = out0
        return gsl00

t = InteractiveSSB(dssb, bf,radius, metadata)
t.show()

radius_data_int : 126 
radius_max_int  : 126 
Dense frame size: 12x 12
sparse_to_dense_datacube_crop_gain_mask dtypes: torch.float32 torch.int32 torch.float32 int64


GridspecLayout(children=(Output(layout=Layout(border='1px solid black', grid_area='widget001'), outputs=({'out…

In [10]:
probe_radius_multiplier = 2

s= t.psi.cpu().numpy()

abss = np.abs(s)
w = abss > abss.max() * 1e-2
t.r_min.max()
probe_radius = np.sqrt(w.sum()*1.0/np.pi) * t.r_min.max()
sampled_radius = int(1.5 * np.ceil(probe_radius))
print(probe_radius,t.r_min.max())

smatrix_bf_radius_big = sampled_radius

radius_factor_big = 2

bin_factor_big = int(np.min(np.ceil(radius/smatrix_bf_radius_big)))

radius_data_int_big = int(np.ceil(radius / bin_factor_big) * bin_factor_big)
radius_max_int_big = int(np.ceil(radius_factor_big * radius / bin_factor_big) * bin_factor_big)
frame_size_big = 2 * radius_max_int_big // bin_factor_big

alpha_max_big = radius_factor_big * metadata.alpha_rad

dx1_big = metadata.wavelength / 2 / alpha_max_big
dxy_big = [dx1_big,dx1_big]

k_max1_big = alpha_max_big / metadata.wavelength
k_max_big = [k_max1_big, k_max1_big]

pixel_step_y = metadata.scan_step[0]/dxy_big[0]
pixel_step_x = metadata.scan_step[1]/dxy_big[1]
pixel_step = np.array([pixel_step_y,pixel_step_x])

print(f'radius: {radius}')
print()
print('=== For iterative reconstruction: ')
print(f'target BF radius    : {smatrix_bf_radius_big}')
print(f'bin_factor          : {bin_factor_big}')
print(f'radius_max_int_big after bin: {radius_max_int_big/bin_factor_big}')
print(f'frame_size after bin: {frame_size_big}')
print(f'rotation (degrees)  : {metadata.rotation_deg}')
print(f'dxy                 : {dx1_big}')
print(f'k_max               : {k_max_big}')
print(f'pixel_step          : {pixel_step}')


fig, ax = plt.subplots()
ax.imshow(w)
AppLayout(center=fig.canvas)

25.711053816447702 0.9942183797899262
radius: 120

=== For iterative reconstruction: 
target BF radius    : 39
bin_factor          : 4
radius_max_int_big after bin: 60.0
frame_size after bin: 120
rotation (degrees)  : 0
dxy                 : 0.5219646493897113
k_max               : [0.9579192778373159, 0.9579192778373159]
pixel_step          : [0.42558018 0.42558018]


AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [11]:
t.C


array([377.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.])

In [12]:
dbig = da[0].crop_symmetric_center(da[0].frame_dimensions/2,  radius*radius_factor_big)

old frames shape: (512, 512, 1146)
new frames shape: (512, 512, 1146)
old frames frame_dimensions: [334 334]
new frames frame_dimensions: [480 480]


In [12]:
dbig

Sparse4DData(scan_dimensions=array([512, 512]), frame_dimensions=array([480, 480]))

In [20]:
ddense = dbig.to_dense(bin_factor_big)

slic_edge = np.s_[:,-5:,:,:]
sds = np.sum(ddense[slic_edge],(0,1)) / np.prod(dssb.scan_dimensions)
vacuum_probe = sds * (sds > sds.max() * 20e-2) 

fig,ax = plt.subplots()
ax.imshow(vacuum_probe)
AppLayout(center=fig.canvas)

radius_data_int : 240 
radius_max_int  : 240 
Dense frame size: 120x 120
sparse_to_dense_datacube_crop_gain_mask dtypes: torch.float32 torch.int32 torch.float32 int64


AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [21]:
max_radius = int(radius_max_int_big/bin_factor_big)
dbig_shifted = dbig.bin(bin_factor_big)
dbig_shifted = dbig_shifted.fftshift_and_pad_to_((2 * max_radius, 2 * max_radius))

s = dbig_shifted.sum_diffraction()
fig,ax = plt.subplots()
ax.imshow(s)
AppLayout(center=fig.canvas)

radius_data_int : 240 
radius_max_int  : 240 
Dense frame size: 120x 120
sparse_to_dense_datacube_crop_gain_mask dtypes: torch.float32 torch.int32 torch.float32 int64
frame_dimensions: [120 120]
scan_dimensions : [512 512]
Using dtype: torch.int16 for indices
Using dtype: torch.uint8 for counts


AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [27]:
metadata.pixel_step = pixel_step
metadata.rotation_deg = t.rotation_deg
metadata.aberrations = t.C.get()
metadata.k_max = k_max_big

metadata.to_h5(results_path / 'ptycho' / f'{scan_number:3d}_dense.h5','meta')
# dbig_shifted.to_h5(results_path / 'ptycho' / f'{scan_number:3d}_dense.h5','data')
metadata

Metadata4D(scan_step=array([0.22213781, 0.22213781], dtype=float32), pixel_step=array([0.42558018, 0.42558018]), k_max=[0.9579192778373159, 0.9579192778373159], alpha_rad=0.02, rotation_deg=0.0, E_ev=80000.0, wavelength=0.041757171951176904, aberrations=array([377.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.]))

In [28]:
t.slic

(slice(207, 457, None), slice(146, 396, None))

In [29]:
h5append(results_path / 'ptycho' / f'{scan_number:3d}_dense.h5',{'dense2':ddense[t.slic]})

In [30]:
metadata.scan_step

array([0.22213781, 0.22213781], dtype=float32)

In [31]:
h5append(results_path / 'ptycho' / f'{scan_number:3d}_dense.h5',{'vacuum':s})

In [32]:
fig,ax = plt.subplots()
ax.imshow(ddense[0,0])
AppLayout(center=fig.canvas)

AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…